In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score
import shap

In [2]:
app_train = pd.read_csv('../data/interim/aplicationtrainlimpio1.csv')
app_test = pd.read_csv('../data/raw/application_ts.csv')
bureau = pd.read_csv('../data/raw/bureau.csv')
bureau_balance = pd.read_csv('../data/raw/bureau_balance.csv')
previous = pd.read_csv('../data/raw/previous_application.csv')
pos_cash = pd.read_csv('../data/raw/POS_CASH_balance.csv')
credit_card = pd.read_csv('../data/raw/credit_card_balance.csv')
installments = pd.read_csv('../data/raw/installments_payments.csv')

In [3]:
bureau_full = bureau.merge(
    bureau_balance,
    on="SK_ID_BUREAU",
    how="left")


In [4]:
bureau_features = (
    bureau_full
    .groupby("SK_ID_CURR")
    .agg(
        bureau_loans=("SK_ID_BUREAU", "nunique"),
        bureau_days_credit_mean=("DAYS_CREDIT", "mean"),
        bureau_days_credit_min=("DAYS_CREDIT", "min"),
        bureau_credit_sum=("AMT_CREDIT_SUM", "sum"),
        bureau_credit_active=("CREDIT_ACTIVE", lambda x: (x == "Active").sum()),
        bureau_months_reported=("MONTHS_BALANCE", "count")
    )
    .reset_index()
)

# Créditos activos actuales (los que más pesan en riesgo)
active_bureau = bureau[bureau["CREDIT_ACTIVE"] == "Active"].groupby("SK_ID_CURR").agg(
    active_loans_count=("SK_ID_BUREAU", "count"),
    active_debt_sum=("AMT_CREDIT_SUM_DEBT", "sum"),
    active_overdue_sum=("AMT_CREDIT_SUM_OVERDUE", "sum")
).reset_index()

# Créditos cerrados recientemente (últimos 2 años)
recent_bureau = bureau[bureau["DAYS_CREDIT"] >= -730].groupby("SK_ID_CURR").agg(
    recent_loans_count=("SK_ID_BUREAU", "count"),
    recent_overdue_mean=("AMT_CREDIT_SUM_OVERDUE", "mean")
).reset_index()

# Unir al dataframe principal de bureau_features
bureau_features = bureau_features.merge(active_bureau, on="SK_ID_CURR", how="left")
bureau_features = bureau_features.merge(recent_bureau, on="SK_ID_CURR", how="left").fillna(0)

bureau_features.to_parquet("../data/interim/bureau_features.parquet")


In [5]:

prev_features = (
    previous
    .groupby("SK_ID_CURR")
    .agg(
        prev_apps=("SK_ID_PREV", "nunique"),
        prev_amt_mean=("AMT_APPLICATION", "mean"),
        prev_amt_max=("AMT_APPLICATION", "max"),
        prev_refused=("NAME_CONTRACT_STATUS", lambda x: (x == "Refused").sum()),
        prev_approved=("NAME_CONTRACT_STATUS", lambda x: (x == "Approved").sum()),
        prev_days_decision_mean=("DAYS_DECISION", "mean")
    )
    .reset_index()
)

prev_features.to_parquet("../data/interim/prev_features.parquet")


In [6]:

# 1. Definir qué es un pago atrasado
installments["late"] = (installments["DAYS_ENTRY_PAYMENT"] > installments["DAYS_INSTALMENT"]).astype(int)
installments["dbd"] = installments["DAYS_INSTALMENT"] - installments["DAYS_ENTRY_PAYMENT"] # Días de anticipación (Days Before Due)

# 2. Agregaciones generales (Toda la historia)
inst_features = installments.groupby("SK_ID_CURR").agg(
    inst_count_total=("SK_ID_PREV", "count"),
    inst_late_ratio_total=("late", "mean"),
    inst_dbd_mean_total=("dbd", "mean")
).reset_index()

# 3. Agregaciones RECIENTES (Último año: 365 días)
# Filtramos primero, luego agrupamos
recent_inst = installments[installments["DAYS_INSTALMENT"] >= -365].groupby("SK_ID_CURR").agg(
    inst_late_ratio_1y=("late", "mean"),
    inst_dbd_mean_1y=("dbd", "mean"),
    inst_amt_paid_1y=("AMT_PAYMENT", "sum")
).reset_index()

# 4. Unir ambas
inst_features = inst_features.merge(recent_inst, on="SK_ID_CURR", how="left").fillna(0)

# 5. Variable de TENDENCIA (¿Está empeorando?)
# Si el ratio de retrasos del último año es mayor al histórico, el riesgo sube.
inst_features['inst_late_trend'] = inst_features['inst_late_ratio_1y'] / (inst_features['inst_late_ratio_total'] + 0.001)
inst_features.to_parquet("../data/interim/installments_features.parquet")


In [7]:

pos_features = (
    pos_cash
    .groupby("SK_ID_CURR")
    .agg(
        pos_loans=("SK_ID_PREV", "nunique"),
        pos_months=("MONTHS_BALANCE", "count"),
        pos_dpd_mean=("SK_DPD", "mean"),
        pos_dpd_def_mean=("SK_DPD_DEF", "mean")
    )
    .reset_index()
)

# DPD máximo en los últimos 6 meses
pos_recent = pos_cash[pos_cash["MONTHS_BALANCE"] >= -6].groupby("SK_ID_CURR").agg(
    pos_recent_max_dpd=("SK_DPD", "max"),
    pos_recent_count_dpd=("SK_DPD", lambda x: (x > 0).sum())
).reset_index()

pos_features = pos_features.merge(pos_recent, on="SK_ID_CURR", how="left").fillna(0)

pos_features.to_parquet("../data/interim/pos_features.parquet")


In [8]:

cc_features = (
    credit_card
    .groupby("SK_ID_CURR")
    .agg(
        cc_loans=("SK_ID_PREV", "nunique"),
        cc_balance_mean=("AMT_BALANCE", "mean"),
        cc_limit_mean=("AMT_CREDIT_LIMIT_ACTUAL", "mean"),
        cc_utilization=("AMT_BALANCE", "mean")
    )
    .reset_index()
)

cc_features.to_parquet("../data/interim/cc_features.parquet")


In [9]:
df_train = (
    app_train
    .merge(bureau_features, on="SK_ID_CURR", how="left")
    .merge(prev_features, on="SK_ID_CURR", how="left")
    .merge(inst_features, on="SK_ID_CURR", how="left")
    .merge(pos_features, on="SK_ID_CURR", how="left")
    .merge(cc_features, on="SK_ID_CURR", how="left")
)

df_train.to_parquet("../data/interim/train_final.parquet")


In [10]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 307511 entries, 0 to 307510
Columns: 108 entries, SK_ID_CURR to cc_utilization
dtypes: float64(56), int64(40), object(12)
memory usage: 253.4+ MB


In [11]:
df_train.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,pos_loans,pos_months,pos_dpd_mean,pos_dpd_def_mean,pos_recent_max_dpd,pos_recent_count_dpd,cc_loans,cc_balance_mean,cc_limit_mean,cc_utilization
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,1.0,19.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,3.0,28.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,1.0,4.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,3.0,21.0,0.0,0.0,0.0,0.0,1.0,0.0,270000.0,0.0
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,5.0,66.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN


In [12]:
TARGET = "TARGET"
y = df_train[TARGET]
X = df_train.drop(columns=[TARGET])

In [13]:
cat_cols = X.select_dtypes(include=["object"]).columns
num_cols = X.select_dtypes(exclude=["object"]).columns

print(f"Categóricas: {len(cat_cols)}")
print(f"Numéricas: {len(num_cols)}")

Categóricas: 12
Numéricas: 95


In [14]:
num_imputer = SimpleImputer(strategy="mean") #change median for mean
cat_imputer = SimpleImputer(strategy="constant", fill_value="missing")

X[num_cols] = num_imputer.fit_transform(X[num_cols])
X[cat_cols] = cat_imputer.fit_transform(X[cat_cols])

In [15]:
X = pd.get_dummies(X, columns=cat_cols, drop_first=True)

In [16]:
print(X.isna().sum().sum())  # must be 0
print(X.shape)

0
(307511, 209)


In [73]:
X_train, X_val, y_train, y_val = train_test_split(
    X,
    y,
    test_size=0.2,
    stratify=y,
    random_state=42)

In [99]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train = pd.DataFrame(
    scaler.fit_transform(X_train),
    columns=X_train.columns,
    index=X_train.index
)

X_val = pd.DataFrame(
    scaler.transform(X_val),
    columns=X_val.columns,
    index=X_val.index
)


In [69]:
'''
This library needs sklearn 1.3.2
and imblearn 0.11.0
'''

from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline as ImbPipeline


# Combine over and under sampling
over = SMOTE(sampling_strategy=0.3)
under = RandomUnderSampler(sampling_strategy=0.7)
steps = [('over', over), ('under', under)]
pipeline = ImbPipeline(steps=steps)
X_train, y_train = pipeline.fit_resample(X_train, y_train)

In [19]:
X_train.shape

(164764, 209)

In [74]:

from sklearn.feature_selection import SelectKBest, f_classif, mutual_info_classif


#F-score (lineal)
selector_f = SelectKBest(score_func=f_classif, k=150)
X_train_f = selector_f.fit_transform(X_train, y_train)

'''
#Mutual Information (no lineal)
selector_mi = SelectKBest(score_func=mutual_info_classif, k=100)
X_train_mi = selector_mi.fit_transform(X_train, y_train)
'''
#Intersection
features_f = X_train.columns[selector_f.get_support()].tolist()
#features_mi = X_train.columns[selector_mi.get_support()].tolist()
#features_common = list(set(features_f) & set(features_mi))


X_train = X_train[features_f]
X_val = X_val[features_f]

print(f"Selected features: {len(features_f)}")

Selected features: 150


In [75]:
neg = (y_train == 0).sum()
pos = (y_train == 1).sum()

scale = neg / pos

In [76]:

params = {
    'objective': 'binary:logistic',
    'eval_metric': 'auc',
    'tree_method': 'hist',

    # core
    'learning_rate': 0.03,
    'n_estimators': 5500,
    'max_depth': 3,
    'min_child_weight': 20,

    # sampling
    'subsample': 0.8,
    'colsample_bytree': 0.8,

    # imbalance
    'scale_pos_weight': scale,

    # regularization
    'reg_alpha': 0.5,
    'reg_lambda': 2.0,
    'gamma': 1.0,
}


xgb_model = XGBClassifier(**params, n_jobs=-1, random_state=42,  early_stopping_rounds=300)

In [77]:
xgb_model.fit(
    X_train,
    y_train,
    eval_set=[(X_val, y_val)],
    verbose=True)


[0]	validation_0-auc:0.69111
[1]	validation_0-auc:0.69366
[2]	validation_0-auc:0.70314
[3]	validation_0-auc:0.70160
[4]	validation_0-auc:0.70688
[5]	validation_0-auc:0.71187
[6]	validation_0-auc:0.71363
[7]	validation_0-auc:0.71887
[8]	validation_0-auc:0.71859
[9]	validation_0-auc:0.71785
[10]	validation_0-auc:0.71803
[11]	validation_0-auc:0.71827
[12]	validation_0-auc:0.72009
[13]	validation_0-auc:0.71956
[14]	validation_0-auc:0.71991
[15]	validation_0-auc:0.72244
[16]	validation_0-auc:0.72273
[17]	validation_0-auc:0.72248
[18]	validation_0-auc:0.72313
[19]	validation_0-auc:0.72334
[20]	validation_0-auc:0.72327
[21]	validation_0-auc:0.72343
[22]	validation_0-auc:0.72356
[23]	validation_0-auc:0.72402
[24]	validation_0-auc:0.72498
[25]	validation_0-auc:0.72522
[26]	validation_0-auc:0.72511
[27]	validation_0-auc:0.72513
[28]	validation_0-auc:0.72543
[29]	validation_0-auc:0.72626
[30]	validation_0-auc:0.72610
[31]	validation_0-auc:0.72613
[32]	validation_0-auc:0.72678
[33]	validation_0-au

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.8, device=None, early_stopping_rounds=300,
              enable_categorical=False, eval_metric='auc', feature_types=None,
              feature_weights=None, gamma=1.0, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.03, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=3,
              max_leaves=None, min_child_weight=20, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=5500,
              n_jobs=-1, num_parallel_tree=None, ...)

In [112]:
'''
#search optim threshold
from sklearn.metrics import roc_curve
import numpy as np

y_val_proba = xgb_model.predict_proba(X_val)[:, 1]
fpr, tpr, thresholds = roc_curve(y_val, y_val_proba)

# finds threshold max (TPR - FPR)
optimal_idx = np.argmax(tpr - fpr)
optimal_threshold = thresholds[optimal_idx]
print(f"Optimal Threshold: {optimal_threshold:.3f}")
'''

'\n#search optim threshold\nfrom sklearn.metrics import roc_curve\nimport numpy as np\n\ny_val_proba = xgb_model.predict_proba(X_val)[:, 1]\nfpr, tpr, thresholds = roc_curve(y_val, y_val_proba)\n\n# finds threshold max (TPR - FPR)\noptimal_idx = np.argmax(tpr - fpr)\noptimal_threshold = thresholds[optimal_idx]\nprint(f"Optimal Threshold: {optimal_threshold:.3f}")\n'

In [80]:
#optimal f1 threshold
from sklearn.metrics import f1_score
import numpy as np

y_val_proba = xgb_model.predict_proba(X_val)[:, 1]

thresholds = np.linspace(0.05, 0.6, 100)
f1_scores = [f1_score(y_val, y_val_proba >= t) for t in thresholds]

optimal_threshold = thresholds[np.argmax(f1_scores)]

print(f"Optimal Threshold (F1): {optimal_threshold:.3f}")


Optimal Threshold (F1): 0.600


In [81]:
from sklearn.metrics import roc_auc_score, classification_report, confusion_matrix, roc_curve

y_val_pred = xgb_model.predict_proba(X_val)[:, 1]
auc = roc_auc_score(y_val, y_val_pred)

adj_threshold = 0.60

y_val_pred = (y_val_pred >= adj_threshold).astype(int)

classification_report = classification_report(y_val, y_val_pred)

#Confusion matrix manual
TP = np.sum((y_val == 1) & (y_val_pred == 1))
FP = np.sum((y_val == 0) & (y_val_pred == 1))
TN = np.sum((y_val == 0) & (y_val_pred == 0))
FN = np.sum((y_val == 1) & (y_val_pred == 0))

FPR = FP / (FP + TN)
Precision = TP / (TP + FP)

print(f"FPR: {FPR:.4f}")
print(f"Precision (PPV): {Precision:.4f}")
print(f"AUC validación: {auc:.5f}")
print(classification_report)

FPR: 0.1601
Precision (PPV): 0.2324
AUC validación: 0.78127
              precision    recall  f1-score   support

           0       0.96      0.84      0.89     56538
           1       0.23      0.55      0.33      4965

    accuracy                           0.82     61503
   macro avg       0.59      0.70      0.61     61503
weighted avg       0.90      0.82      0.85     61503

